# Preparar clasificación de noticias RPP
Este cuaderno:
1. Cargamos las 50 noticias desde `data/rpp_source.json`.
2. Generamos el bloque numerado para pedir clasificación a un LLM (0-World, 1-Sports, 2-Business, 3-Science/Tech).
3. Guardamos el resultado clasificado en `data/rpp_classified.json`.

In [ ]:
import json, pandas as pd, os

# === 1. Cargar las 50 noticias del RSS ===
with open('data/rpp_source.json', 'r', encoding='utf-8') as f:
    news = json.load(f)
df = pd.DataFrame(news)
print(f'Noticias cargadas: {len(df)}')
df.head(2)

In [ ]:
# === 2. Generar bloque numerado ===
for i, row in enumerate(df.itertuples(), start=1):
    print(f"{i}) {row.title.strip()} — {row.description.strip()}\n")

print('\n Copiamos este bloque y lo pegamos en nuestra LLM (ChatGPT o modelo local) con la instrucción:')
print('Clasifica cada noticia en una categoría AG News: 0=World, 1=Sports, 2=Business, 3=Science/Technology.')

In [ ]:
# === 3. Guardar archivo clasificado (cuando tengas las etiquetas del LLM) ===
# Sustituye los valores de 'labels' con la lista que devuelva el modelo
labels = []  # ← pega aquí la lista, ej: [0,1,2,0,3,...]

assert len(labels) == len(df), 'La lista de etiquetas debe tener 50 elementos.'

df['label_llm'] = labels
os.makedirs('data', exist_ok=True)
out_path = 'data/rpp_classified.json'
df.to_json(out_path, orient='records', indent=2, force_ascii=False)
print(f'Guardado {out_path} con {len(df)} registros.')